<a href="https://colab.research.google.com/github/connorpheraty/DS-Project-2/blob/master/DS3_Project2_TWP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tanzania Water Pumps - Kaggle Challenge

Using data from Taarifa and the Tanzanian Ministry of Water, can you predict which pumps are functional, which need some repairs, and which don't work at all? Predict one of these three classes based on a number of variables about what kind of pump is operating, when it was installed, and how it is managed. A smart understanding of which waterpoints will fail can improve maintenance operations and ensure that clean, potable water is available to communities across Tanzania.

## Import Data 

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)

In [4]:
train = pd.read_csv('Data/train_features.csv', index_col='id')
test = pd.read_csv('Data/test.csv', index_col='id')
target = pd.read_csv('Data/train_labels.csv', index_col='id')

### Check dtypes, cardinality, and null values

In [5]:
# Check data types of features
train.dtypes

amount_tsh               float64
date_recorded             object
funder                    object
gps_height                 int64
installer                 object
longitude                float64
latitude                 float64
wpt_name                  object
num_private                int64
basin                     object
subvillage                object
region                    object
region_code                int64
district_code              int64
lga                       object
ward                      object
population                 int64
public_meeting            object
recorded_by               object
scheme_management         object
scheme_name               object
permit                    object
construction_year          int64
extraction_type           object
extraction_type_group     object
extraction_type_class     object
management                object
management_group          object
payment                   object
payment_type              object
water_qual

In [6]:
# Check cardinality
train.nunique()

amount_tsh                  98
date_recorded              356
funder                    1897
gps_height                2428
installer                 2145
longitude                57516
latitude                 57517
wpt_name                 37400
num_private                 65
basin                        9
subvillage               19287
region                      21
region_code                 27
district_code               20
lga                        125
ward                      2092
population                1049
public_meeting               2
recorded_by                  1
scheme_management           12
scheme_name               2696
permit                       2
construction_year           55
extraction_type             18
extraction_type_group       13
extraction_type_class        7
management                  12
management_group             5
payment                      7
payment_type                 7
water_quality                8
quality_group                6
quantity

In [7]:
# Check for null values
train.isnull().sum()

amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_group                0
quantity

## Preprocessing and Feature Engineering

In [8]:
def wrangle_data(df):
    
    # Drop non-relevant columns
    # Features here either share high covariance with other features 
    # Or are not highly correlated with target variables
    df = df.drop(['wpt_name', 'num_private', 'region', 'district_code', 'ward', 'public_meeting',
                    'recorded_by', 'scheme_name', 'funder', 'subvillage', 'extraction_type_group',
                    'extraction_type_class', 'waterpoint_type_group', 'payment', 'management_group',
                   'quality_group', 'installer'], axis=1)
    
    # region_code --> categorical
    df['region_code'] = df['region_code'].astype(str)
    
    # date_recorded --> split by year, month, and day
    df[['year', 'month', 'day']] = df['date_recorded'].str.split(pat='-', n=-1, expand=True)
    
    # drop date_recorded
    df = df.drop('date_recorded', axis=1)
    
    # Transform year, month, and day into integers
    df[['year', 'month', 'day']] = df[['year', 'month', 'day']].astype(int)
    
    # Change all construction years with zero values to year of record
    df['construction_year'].loc[df['construction_year'] == 0] = df['year']
    
    # Split Data into seperate Numeric and Categorical dataframes
    def split(df):
        numerics = ['int64', 'float64']
        df_num = df.select_dtypes(include=numerics)
        df_cat = df.drop(df_num, axis='columns')
        print(df.shape, df_num.shape, df_cat.shape)
        return df_num, df_cat
    
    df_num, df_cat = split(df)
    
    #-------------- Feature Engineering ------------------#
    
    # Create 'age at inspection' feature subtracting the year inspected from the year constructed
    df_num['age_at_inspection'] = df_cat['year'] - df_num['construction_year']
    
    # 
    mean_lat_train = df_num['latitude'].mean()
    mean_long_train = df_num['longitude'].mean()
    df_num['distance_x_height'] = np.sqrt((df_num['gps_height']**2 + df_num['longitude'] - mean_long_train)**2 + (df_num['latitude'] - mean_lat_train)**2)
    
    df_cat = df_cat.drop(['year', 'month', 'day'], axis=1)
    df_num = df_num.drop('construction_year', axis=1)
    
    return df_num, df_cat

In [9]:
train_num, train_cat = wrangle_data(train)
test_num, test_cat = wrangle_data(test)

c:\users\clemster\documents\cours\epsi\tanzania_water_kaggle_competition\env\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
c:\users\clemster\documents\cours\epsi\tanzania_water_kaggle_competition\env\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\clemster\documents\cours\epsi\tanzania_water_kaggle_competition\env\lib\site-packages\ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

(59400, 24) (59400, 6) (59400, 18)
(14850, 24) (14850, 6) (14850, 18)


## Merge Numeric and Categorical DataFrames

In [10]:
# One-Hot-Encodes category dataframe
def dummy_df(category_df):
    df_dummy = pd.DataFrame()
    for col in category_df.columns:
        df_dummy = pd.concat([df_dummy, pd.get_dummies(category_df[col], drop_first=True, prefix = 'Is')], axis='columns')
    return df_dummy

In [11]:
# Run function on train and test df's
df_dumb_train = dummy_df(train_cat)
df_dumb_test = dummy_df(test_cat)

In [12]:
# Check new shape
df_dumb_train.shape, df_dumb_test.shape

((59400, 242), (14850, 239))

In [13]:
# create list of columns for both dataframes
train_cols = list(df_dumb_train.columns)
test_cols = list(df_dumb_test.columns)

In [14]:
# Align dataframes  
train, test = df_dumb_train.align(df_dumb_test, join='left', axis=1)

In [15]:
# Verify alignment
train.shape, test.shape

((59400, 332), (14850, 332))

In [16]:
# Eliminate nan values
test = test.replace(np.nan, 0)

In [17]:
# Concatenate numeric and categorical dataframes into one
X_train = pd.concat([train_num, train], axis=1)
X_test = pd.concat([test_num, test], axis=1)

In [18]:
X_train.head()

,amount_tsh,gps_height,longitude,latitude,population,age_at_inspection,distance_x_height,Is_10,Is_11,Is_12,Is_13,Is_14,Is_15,Is_16,Is_17,Is_18,Is_19,Is_2,Is_20,Is_21,Is_24,Is_3,Is_4,Is_40,Is_5,Is_6,Is_60,Is_7,Is_8,Is_80,Is_9,Is_90,Is_99,Is_Arusha Urban,Is_Babati,Is_Bagamoyo,Is_Bahi,Is_Bariadi,Is_Biharamulo,Is_Bukoba Rural,Is_Bukoba Urban,Is_Bukombe,Is_Bunda,Is_Chamwino,Is_Chato,Is_Chunya,Is_Dodoma Urban,Is_Geita,Is_Hai,Is_Hanang,Is_Handeni,Is_Igunga,Is_Ilala,Is_Ileje,Is_Ilemela,Is_Iramba,Is_Iringa Rural,Is_Kahama,Is_Karagwe,Is_Karatu,Is_Kasulu,Is_Kibaha,Is_Kibondo,Is_Kigoma Rural,Is_Kigoma Urban,Is_Kilindi,Is_Kilolo,Is_Kilombero,Is_Kilosa,Is_Kilwa,Is_Kinondoni,Is_Kisarawe,Is_Kishapu,Is_Kiteto,Is_Kondoa,Is_Kongwa,Is_Korogwe,Is_Kwimba,Is_Kyela,Is_Lake Nyasa,Is_Lake Rukwa,Is_Lake Tanganyika,Is_Lake Victoria,Is_Lindi Rural,Is_Lindi Urban,Is_Liwale,Is_Longido,Is_Ludewa,Is_Lushoto,Is_Mafia,Is_Magu,Is_Makete,Is_Manyoni,Is_Masasi,Is_Maswa,Is_Mbarali,Is_Mbeya Rural,Is_Mbinga,Is_Mbozi,Is_Mbulu,Is_Meatu,Is_Meru,Is_Misenyi,Is_Missungwi,Is_Mkinga,Is_Mkuranga,Is_Monduli,Is_Morogoro Rural,Is_Morogoro Urban,Is_Moshi Rural,Is_Moshi Urban,Is_Mpanda,Is_Mpwapwa,Is_Mtwara Rural,Is_Mtwara Urban,Is_Mufindi,Is_Muheza,Is_Muleba,Is_Musoma Rural,Is_Mvomero,Is_Mwanga,Is_Nachingwea,Is_Namtumbo,Is_Nanyumbu,Is_Newala,Is_Ngara,Is_Ngorongoro,Is_Njombe,Is_Nkasi,Is_None,Is_Nyamagana,Is_Nzega,Is_Other,Is_Pangani,Is_Pangani,Is_Pangani,Is_Pangani,Is_Parastatal,Is_Private operator,Is_Rombo,Is_Rorya,Is_Ruangwa,Is_Rufiji,Is_Rufiji,Is_Rufiji,Is_Rufiji,Is_Rungwe,Is_Ruvuma / Southern Coast,Is_SWC,Is_Same,Is_Sengerema,Is_Serengeti,Is_Shinyanga Rural,Is_Shinyanga Urban,Is_Siha,Is_Sikonge,Is_Simanjiro,Is_Singida Rural,Is_Singida Urban,Is_Songea Rural,Is_Songea Urban,Is_Sumbawanga Rural,Is_Sumbawanga Urban,Is_Tabora Urban,Is_Tandahimba,Is_Tanga,Is_Tarime,Is_Temeke,Is_True,Is_Trust,Is_Tunduru,Is_Ukerewe,Is_Ulanga,Is_Urambo,Is_Uyui,Is_VWC,Is_WUA,Is_WUG,Is_Wami / Ruvu,Is_Water Board,Is_Water authority,Is_cemo,Is_climax,Is_communal standpipe,Is_communal standpipe multiple,Is_dam,Is_dam,Is_dam,Is_dam,Is_enough,Is_enough,Is_enough,Is_enough,Is_fluoride,Is_fluoride abandoned,Is_gravity,Is_hand dtw,Is_hand pump,Is_improved spring,Is_india mark ii,Is_india mark iii,Is_insufficient,Is_insufficient,Is_insufficient,Is_insufficient,Is_ksb,Is_lake,Is_machine dbh,Is_milky,Is_mono,Is_monthly,Is_never pay,Is_nira/tanira,Is_on failure,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other,Is_other - mkulima/shinyanga,Is_other - play pump,Is_other - rope pump,Is_other - school,Is_other - swn 81,Is_parastatal,Is_per bucket,Is_private operator,Is_rainwater harvesting,Is_rainwater harvesting,Is_rainwater harvesting,Is_rainwater harvesting,Is_river,Is_river/lake,Is_salty,Is_salty abandoned,Is_seasonal,Is_seasonal,Is_seasonal,Is_seasonal,Is_shallow well,Is_shallow well,Is_shallow well,Is_shallow well,Is_soft,Is_spring,Is_spring,Is_spring,Is_spring,Is_submersible,Is_surface,Is_swn 80,Is_trust,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_unknown,Is_vwc,Is_walimi,Is_water authority,Is_water board,Is_windmill,Is_wua,Is_wug
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

In [19]:
# Verify elimination of nan values
np.any(np.isnan(X_train)), np.any(np.isnan(X_test))

(False, False)

In [20]:
# View new shape
X_train.shape, X_test.shape

((59400, 339), (14850, 339))

## Random Forest Classifier

The Random Forest Classifier provided me with my highest accuracy score.

### Transform target dataframe from 0,1,2 to 'functional', 'non functional', 'functional needs repair'

In [21]:
target.head()

,status_group
id,
69572,0
8776,0
34310,0
67743,1
19728,0


In [22]:
# Function to map proper labels
def label_mapper(y):
    
    if y == 0:
        return 'functional'
    if y == 1:
        return 'non functional'
    if y == 2:
        return 'functional needs repair'

In [23]:
y_train = target['status_group']
y_train = y_train.map(label_mapper)
y_train.head()

id
69572        functional
8776         functional
34310        functional
67743    non functional
19728        functional
Name: status_group, dtype: object

### Split Data into Train and Val

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)

X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape

((47520, 339), (11880, 339), (14850, 339), (47520,), (11880,))

### Create Random Forest Classifier Model

In [25]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import StandardScaler


model = RandomForestClassifier(n_estimators=1000, 
                                max_depth=30,
                                min_samples_split = 12,
                                min_samples_leaf = 1,
                                criterion = 'gini', 
                                max_features = 6, 
                                oob_score = True, 
                                random_state=42)

In [26]:
# Scale numeric data using the standard scaler
scaler = StandardScaler()
X_train_rf = scaler.fit_transform(X_train)
X_test_rf = scaler.transform(X_test)

model.fit(X_train_rf, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=30, max_features=6, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=12,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=True, random_state=42, verbose=0,
                       warm_start=False)

In [27]:
# Predict y_train and y_test
y_pred_train = model.predict(X_train_rf)
y_pred = model.predict(X_test_rf)

### Cross Validate Model

In [28]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score

scores = cross_validate(model,
                        X_train_rf,y_train, 
                        scoring = 'accuracy', cv=5) 

In [29]:
pd.DataFrame(scores)
scores

{'fit_time': array([70.84256935, 67.39185286, 67.94730783, 68.78213739, 71.83394384]),
 'score_time': array([2.48535466, 2.34074283, 2.37564802, 2.39759326, 2.52225924]),
 'test_score': array([0.79581317, 0.7932667 , 0.79543302, 0.79827423, 0.78617279])}

## XGBoost

In [30]:
from xgboost import XGBClassifier
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [31]:
scaler = StandardScaler()
X_train_xg = scaler.fit_transform(X_train)
X_test_xg = scaler.transform(X_test)
X_val_xg = scaler.transform(X_val)


In [ ]:
grboost = XGBClassifier(learning_rate=0.07, booster = 'dart', n_estimators=200, max_depth = 8)
grboost.fit(X_train_rf, y_train)

In [ ]:
y_train_pred_boost = grboost.predict(X_test_xg)

In [ ]:
y_val_pred_boost = grboost.predict(X_val_xg, ntree_limit=50)

In [ ]:
accuracy_score(y_train, y_train_pred_boost)

In [ ]:
accuracy_score(y_val, y_val_pred_boost)

In [ ]:
print(len(X_test_xg))
print(len(X_val_xg))

## Creating a Submission

In [ ]:
print(len(y_val_pred_boost))
# Function to create submission in proper format
test = pd.read_csv('Data/test.csv')
def create_submission(var):
    submission = pd.DataFrame({
        'id': test['id'],
        'status_group': var
    })
    submission.to_csv('sending_boost.csv', index=False, sep=',')

In [ ]:
create_submission(y_pred)

In [ ]:
y_pred_df = pd.DataFrame(y_pred)

In [ ]:
sample_submission = pd.read_csv('Data/test_features.csv')
submission = sample_submission.copy()
submission['status_group'] = y_pred

In [ ]:
submission.head()

In [ ]:
submission.status_group.value_counts(normalize=True)